In [1]:
!pip install transformers datasets



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [8]:
from datasets import load_dataset

# Load first 1000 examples from Arabic OSCAR dataset (fast training)
dataset = load_dataset("oscar", "unshuffled_deduplicated_ar", split="train[:100]")

# Filter out very short samples
dataset = dataset.filter(lambda example: len(example["text"]) > 20)


Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [9]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Prevents padding errors

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [10]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


Embedding(50257, 768)

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results_arabic_fast",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=1,
    logging_steps=10,
    logging_dir="./logs_arabic",
)


In [13]:
from transformers import Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()


Step,Training Loss
10,2.587700
20,2.537700
30,2.464600
40,2.446300
50,2.488800


TrainOutput(global_step=50, training_loss=2.505022201538086, metrics={'train_runtime': 292.0117, 'train_samples_per_second': 0.342, 'train_steps_per_second': 0.171, 'total_flos': 6532300800000.0, 'train_loss': 2.505022201538086, 'epoch': 1.0})

In [14]:
model.save_pretrained("./gpt2_arabic_demo")
tokenizer.save_pretrained("./gpt2_arabic_demo")


('./gpt2_arabic_demo/tokenizer_config.json',
 './gpt2_arabic_demo/special_tokens_map.json',
 './gpt2_arabic_demo/vocab.json',
 './gpt2_arabic_demo/merges.txt',
 './gpt2_arabic_demo/added_tokens.json')

In [16]:
from transformers import pipeline

generator = pipeline("text-generation", model="./gpt2_arabic_demo", tokenizer="./gpt2_arabic_demo")

# Example Arabic prompt
prompt = "الذكاء الاصطناعي هو"
result = generator(prompt, max_length=50, num_return_sequences=1)

print(result[0]["generated_text"])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


الذكاء الاصطناعي هواض المنت ٱاساتا؂ الخبين عحزيدم ك
